In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

In [36]:
def extract_fishing_points(url):
    """
    지도 사이트에서 낚시 포인트 이름을 추출하는 함수
    
    Args:
        url (str): 낚시 포인트 지도가 있는 웹사이트 URL
    
    Returns:
        list: 낚시 포인트 이름 리스트
    """
    # Chrome 옵션 설정
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 브라우저 창 표시 없이 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # WebDriver 설정
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    fishing_points = []
    
    try:
        # 웹페이지 로드
        driver.get(url)
        print("페이지 로딩 중...")
        
        # 페이지가 완전히 로드될 때까지 잠시 대기
        time.sleep(3)
        
        # 지도가 완전히 로드될 때까지 기다림 (예: 특정 요소가 나타날 때까지)
        wait = WebDriverWait(driver, 20)
        
        # 지도가 로드될 때까지 기다림
        wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.label"))
        )
        
        # 축소 버튼 찾기 (title="축소" 및 type="button" 속성을 가진 버튼)
        try:
            zoom_out_button = driver.find_element(By.XPATH, '//button[@title="축소" and @type="button"]')
            
            # 최대한 축소하기 위해 여러 번 클릭 (10번 정도면 최대 축소에 충분할 것)
            print("지도 최대 축소 중...")
            for _ in range(10):
                try:
                    zoom_out_button.click()
                    time.sleep(0.5)  # 각 클릭 사이에 잠시 대기
                except Exception as e:
                    print(f"더 이상 축소할 수 없습니다: {e}")
                    break
            
            print("지도 축소 완료")
            
            # 지도가 업데이트될 때까지 잠시 대기
            time.sleep(2)
            
        except Exception as e:
            print(f"축소 버튼을 찾을 수 없습니다: {e}")
        
        # div(class="label") 안에 span(class="center") 요소 찾기
        point_elements = driver.find_elements(By.CSS_SELECTOR, "div.label span.center")
        
        # 각 포인트 이름 추출
        for point in point_elements:
            point_name = point.text.strip()
            if point_name:  # 빈 문자열이 아닌 경우만 추가
                fishing_points.append({"name": point_name})
        
        print(f"총 {len(fishing_points)}개의 낚시 포인트를 찾았습니다.")
        
    except Exception as e:
        print(f"데이터 추출 중 오류 발생: {e}")
    
    finally:
        # WebDriver 종료
        driver.quit()
    
    return fishing_points

def save_to_csv(fishing_points, filename):
    try:
        filename = filename + ".csv"
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['name']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for point in fishing_points:
                writer.writerow(point)
                
        print(f"{len(fishing_points)}개의 낚시 포인트 정보를 {filename}에 저장했습니다.")
        
    except Exception as e:
        print(f"CSV 파일 저장 중 오류 발생: {e}")


# 낚시 포인트 추출 실행
west_sea_url = "https://m.badatime.com/time_map.jsp?idx=121&menu=2"
south_sea_url = "https://m.badatime.com/time_map.jsp?idx=41&menu=2"
east_sea_url = "https://m.badatime.com/time_map.jsp?idx=203&menu=2"
jeju_sea_url = "https://m.badatime.com/time_map.jsp?idx=67&menu=2"

west_sea_points = extract_fishing_points(west_sea_url)

south_sea_points = extract_fishing_points(south_sea_url)

east_sea_points = extract_fishing_points(east_sea_url)

jeju_sea_points = extract_fishing_points(jeju_sea_url)

페이지 로딩 중...
지도 최대 축소 중...
지도 축소 완료
총 537개의 낚시 포인트를 찾았습니다.
페이지 로딩 중...
지도 최대 축소 중...
지도 축소 완료
총 731개의 낚시 포인트를 찾았습니다.
페이지 로딩 중...
지도 최대 축소 중...
지도 축소 완료
총 64개의 낚시 포인트를 찾았습니다.
페이지 로딩 중...
지도 최대 축소 중...
지도 축소 완료
총 83개의 낚시 포인트를 찾았습니다.


In [37]:
print(type(west_sea_points))
fishing_points = west_sea_points + east_sea_points + south_sea_points + jeju_sea_points
print(len(fishing_points))
save_to_csv(fishing_points, filename="fishing_points")

<class 'list'>
1415
1415개의 낚시 포인트 정보를 fishing_points.csv에 저장했습니다.


In [ ]:
import requests
import zipfile
import io
import os
import csv
import json
import time
from haversine import haversine
import numpy as np

def download_coastline_data():
    """
    자동으로 해안선 데이터를 다운로드하거나 로컬에서 로드하는 함수
    
    Returns:
        list: 해안선 좌표 리스트 [(lat1, lon1), (lat2, lon2), ...]
    """
    # 데이터 캐시 파일 경로
    cache_file = "coastline_data.json"
    
    # 캐시 파일이 있으면 로드
    if os.path.exists(cache_file):
        try:
            with open(cache_file, 'r', encoding='utf-8') as f:
                coastline_data = json.load(f)
                print(f"해안선 데이터를 캐시에서 로드했습니다. 총 {len(coastline_data)} 개의 좌표.")
                return [(float(lat), float(lon)) for lat, lon in coastline_data]
        except Exception as e:
            print(f"캐시 파일 로드 중 오류: {e}")
            print("다시 다운로드를 시도합니다.")
    
    # 상세 해안선 데이터 생성 (간략화된 버전)
    coastline_data = generate_detailed_coastline()
    
    # 캐시 파일로 저장
    try:
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(coastline_data, f)
        print(f"해안선 데이터를 캐시 파일 '{cache_file}'에 저장했습니다.")
    except Exception as e:
        print(f"캐시 파일 저장 중 오류: {e}")
    
    return [(float(lat), float(lon)) for lat, lon in coastline_data]

def generate_detailed_coastline():
    """
    한국 해안선의 상세 좌표를 생성하는 함수
    (실제로는 공공 데이터를 다운로드해야 하지만, 여기서는 간략화된 버전으로 구현)
    
    Returns:
        list: 해안선 좌표 리스트 [(lat1, lon1), (lat2, lon2), ...]
    """
    coastline_data = []
    
    # 동해안
    east_coast_base = [
        (38.6078, 128.3836),  # 강원도 고성
        (38.2074, 128.5933),  # 강원도 속초
        (37.7773, 128.9478),  # 강원도 강릉
        (37.4913, 129.1813),  # 강원도 동해
        (37.5036, 129.4327),  # 강원도 삼척
        (36.9932, 129.4175),  # 경북 울진
        (36.5356, 129.4454),  # 경북 영덕
        (36.0541, 129.3840),  # 경북 포항
        (35.8907, 129.5126),  # 경북 경주
        (35.5951, 129.4476),  # 울산
        (35.1545, 129.1499),  # 부산 해운대
        (35.0961, 129.0335)   # 부산 남구
    ]
    
    # 남해안
    south_coast_base = [
        (35.0961, 129.0335),  # 부산 남구
        (34.9406, 128.8219),  # 부산 가덕도
        (34.8118, 127.9068),  # 경남 통영
        (34.9348, 128.0428),  # 경남 거제
        (34.7700, 127.6639),  # 전남 여수
        (34.5645, 127.3075),  # 전남 고흥
        (34.6182, 127.0758),  # 전남 보성
        (34.3052, 126.5375),  # 전남 완도
        (34.3182, 126.3222),  # 전남 해남
        (34.6813, 125.9531),  # 전남 진도
        (34.4848, 126.2637),  # 전남 해남
        (34.2892, 126.1630),  # 전남 완도군 청산도
        (34.1523, 126.0845)   # 전남 완도군 소안도
    ]
    
    # 서해안
    west_coast_base = [
        (34.1523, 126.0845),  # 전남 완도군 소안도
        (34.6735, 125.4137),  # 전남 신안군 가거도
        (34.9833, 125.3333),  # 전남 신안군 흑산도
        (35.1695, 125.9881),  # 전남 목포
        (35.7364, 126.2642),  # 전북 부안
        (35.9598, 126.6014),  # 전북 군산
        (36.3285, 126.5926),  # 충남 보령
        (36.7817, 126.1494),  # 충남 태안
        (37.0255, 126.4090),  # 충남 당진
        (37.1749, 126.2357),  # 경기 화성
        (37.4462, 126.2626),  # 경기 시흥
        (37.5456, 126.5906),  # 인천 영종도
        (37.3804, 126.5973),  # 인천 강화
        (37.6842, 126.2872)   # 인천 북부
    ]
    
    # 제주도
    jeju_base = [
        (33.4996, 126.5312),  # 제주시
        (33.2496, 126.5628),  # 서귀포시
        (33.5219, 126.8567),  # 성산일출봉
        (33.4485, 126.9184),  # 우도
        (33.2285, 126.2143),  # 차귀도
        (33.3936, 126.2412),  # 비양도
        (33.1975, 126.2659),  # 마라도
        (33.1365, 126.5101)   # 가파도
    ]
    
    # 울릉도/독도
    ulleung_dokdo_base = [
        (37.5042, 130.8696),  # 울릉도
        (37.2424, 131.8642)   # 독도
    ]
    
    # 인접 점 사이에 더 많은 점 생성하여 해안선 상세화
    all_base_points = [east_coast_base, south_coast_base, west_coast_base, jeju_base, ulleung_dokdo_base]
    
    for region in all_base_points:
        # 각 지역의 해안선 좌표 생성
        for i in range(len(region)-1):
            start_lat, start_lon = region[i]
            end_lat, end_lon = region[i+1]
            
            # 두 지점 사이의 거리에 따라 중간 지점 개수 결정
            distance = haversine((start_lat, start_lon), (end_lat, end_lon))
            num_points = max(2, int(distance * 5))  # 1km당 약 5개 지점
            
            # 선형 보간으로 중간 지점 생성
            for j in range(num_points+1):
                ratio = j / num_points
                lat = start_lat + ratio * (end_lat - start_lat)
                lon = start_lon + ratio * (end_lon - start_lon)
                coastline_data.append((lat, lon))
    
    # 근해 섬 추가
    islands = [
        # 서해 섬들
        (37.2460, 126.4847),  # 영흥도
        (35.8143, 126.4201),  # 위도
        (36.9188, 126.2878),  # 원산도
        (36.3779, 126.4297),  # 안면도
        (37.5836, 125.9817),  # 백령도
        (37.6781, 125.7069),  # 대청도
        (37.7459, 125.7906),  # 소청도
        (36.9188, 125.7906),  # 덕적도
        
        # 남해 섬들
        (34.8376, 128.4395),  # 남해 창선도
        (34.7163, 127.5222),  # 돌산도
        (34.2033, 127.2865),  # 금오도
        (34.6025, 125.4215),  # 홍도
        (34.2841, 127.1688),  # 연도
        
        # 동해 섬들
        (36.4128, 129.4707),  # 죽변
        (35.3791, 129.3890)   # 기장
    ]
    
    # 섬 주변 좌표 생성
    for island_lat, island_lon in islands:
        # 섬 주변에 8방향 지점 추가
        for angle in range(0, 360, 45):
            rad = np.radians(angle)
            offset = 0.01  # 대략 1km 정도의 거리
            lat = island_lat + offset * np.sin(rad)
            lon = island_lon + offset * np.cos(rad)
            coastline_data.append((lat, lon))
    
    print(f"총 {len(coastline_data)}개의 해안선 좌표를 생성했습니다.")
    return coastline_data

def is_near_sea(latitude, longitude, sea_coords, max_distance_km=5.0):
    """
    주어진 좌표가 바다 근처인지 확인하는 함수
    
    Args:
        latitude (float): 위도
        longitude (float): 경도
        sea_coords (list): 바다 해안선 좌표 리스트 [(lat1, lon1), (lat2, lon2), ...]
        max_distance_km (float): 바다로부터 최대 거리 (km)
    
    Returns:
        bool: 바다 근처인 경우 True, 아니면 False
    """
    point = (latitude, longitude)
    
    # 해안선 좌표가 많은 경우, 효율성을 위해 인접한 좌표만 먼저 확인
    # 위도 기준으로 탐색 범위 제한 (대략적으로 계산)
    lat_diff = max_distance_km / 111.0  # 1도 = 약 111km
    
    # 빠른 필터링: 좌표의 위도/경도 범위로 첫 필터링
    nearby_coords = [
        sea_point for sea_point in sea_coords
        if abs(sea_point[0] - latitude) <= lat_diff and
           abs(sea_point[1] - longitude) <= lat_diff * 1.5  # 경도는 위도보다 거리 변화가 큼
    ]
    
    # 인접 좌표가 있으면 정확한 거리 계산
    if nearby_coords:
        for sea_point in nearby_coords:
            distance = haversine(point, sea_point)
            if distance <= max_distance_km:
                return True
    
    return False

def is_coastal_location(location_name):
    """
    위치 이름을 기반으로 해안가일 가능성을 판단하는 함수
    
    Args:
        location_name (str): 위치 이름
    
    Returns:
        bool: 해안가일 가능성이 높으면 True, 아니면 False
    """
    coastal_keywords = [
        '항', '포', '도', '진', '해변', '해수욕장', '바다', '어촌', 
        '해안', '갯벌', '선착장', '방파제', '등대', '요트', '마리나',
        '배', '페리', '여객터미널', '수산', '어판장', '어시장', '수협',
        '바닷가', '해', '부두', '간석지', '갑문', '등표', '방조제'
    ]
    
    for keyword in coastal_keywords:
        if keyword in location_name:
            return True
    
    return False

def load_administrative_coastal_areas():
    """
    한국의 해안가 행정구역 리스트를 반환하는 함수
    
    Returns:
        list: 해안가 행정구역 리스트
    """
    # 해안가에 접한 시군구 목록
    coastal_areas = [
        # 서해안
        '인천', '강화', '옹진', '중구', '서구', '동구', '남구', '연수구', '남동구', '김포', '화성', 
        '안산', '시흥', '평택', '당진', '서산', '태안', '보령', '서천', '군산', '김제', '부안', 
        '고창', '영광', '함평', '무안', '신안', '목포', '해남', '진도', '완도',
        
        # 남해안
        '여수', '순천', '광양', '고흥', '보성', '장흥', '강진', '완도', '고성', '통영', '거제', 
        '남해', '하동', '사천', '진주', '창원', '김해', '부산', '기장',
        
        # 동해안
        '울산', '경주', '포항', '영덕', '울진', '삼척', '동해', '강릉', '양양', '속초', '고성',
        
        # 제주
        '제주', '서귀포'
    ]
    
    return coastal_areas

def filter_fishing_points_by_location(fishing_points):
    """
    낚시 포인트 이름을 기반으로 해안가일 가능성이 높은 포인트를 필터링하는 함수
    
    Args:
        fishing_points (list): 낚시 포인트 이름 리스트
    
    Returns:
        list: 해안가일 가능성이 높은 낚시 포인트 이름 리스트
    """
    coastal_areas = load_administrative_coastal_areas()
    result = []
    
    for point in fishing_points:
        # 포인트 이름에 해안 키워드가 있거나, 해안가 행정구역 이름이 포함된 경우
        if is_coastal_location(point) or any(area in point for area in coastal_areas):
            result.append(point)
    
    print(f"이름 기반 필터링: {len(fishing_points)}개 중 {len(result)}개가 해안가일 가능성이 높습니다.")
    return result

def process_fishing_points_improved(input_filename, output_filename, client_id, client_secret, max_distance_km=5.0):
    """
    낚시 포인트 이름을 좌표로 변환하고 바다 근처 포인트만 필터링하는 함수 (개선된 버전)
    
    Args:
        input_filename (str): 낚시 포인트 이름이 저장된 CSV 파일 경로
        output_filename (str): 결과를 저장할 CSV 파일 경로
        client_id (str): 네이버 API 클라이언트 ID
        client_secret (str): 네이버 API 클라이언트 시크릿
        max_distance_km (float): 바다로부터 최대 거리 (km)
    """
    # 낚시 포인트 이름 로드
    fishing_points = load_fishing_points(input_filename)
    
    if not fishing_points:
        print("처리할 낚시 포인트가 없습니다.")
        return
    
    # 이름 기반으로 해안가일 가능성이 높은 포인트 필터링
    coastal_points = filter_fishing_points_by_location(fishing_points)
    
    # 상세 해안선 좌표 로드
    sea_coords = download_coastline_data()
    print(f"{len(sea_coords)}개의 해안선 좌표를 로드했습니다.")
    
    # 결과 저장할 리스트
    results = []
    
    # 각 낚시 포인트에 대해 좌표 검색 및 바다 근처 확인
    for i, point_name in enumerate(coastal_points):
        print(f"[{i+1}/{len(coastal_points)}] '{point_name}' 처리 중...")
        
        # 검색 키워드 준비
        search_keywords = [point_name]
        
        # 키워드가 없는 경우에만 추가 검색어 생성
        if not is_coastal_location(point_name):
            search_keywords.extend([
                point_name + " 낚시",
                point_name + " 포구",
                point_name + " 해변",
                point_name + " 항구"
            ])
        
        # 검색어 목록을 순회하며 좌표 찾기
        result = None
        
        # 네이버 API로 좌표 검색 함수 (별도로 구현 필요)
        for keyword in search_keywords:
            # 이 함수는 별도로 구현해야 함
            # result = get_coordinates_from_naver(keyword, client_id, client_secret)
            # 아래는 예시 코드
            # ...
            
            if result:
                result["name"] = point_name  # 원래 이름으로 복원
                result["search_keyword"] = keyword  # 사용된 검색어 저장
                break
        
        if result:
            # 바다 근처인지 확인
            is_sea_nearby = is_near_sea(result["latitude"], result["longitude"], sea_coords, max_distance_km)
            result["near_sea"] = is_sea_nearby
            
            if is_sea_nearby:
                print(f"  ✓ 바다 근처 포인트: {point_name} ({result['latitude']}, {result['longitude']})")
                results.append(result)
            else:
                print(f"  ✗ 바다에서 멀리 떨어짐: {point_name}")
        else:
            print(f"  ! 좌표를 찾을 수 없음: {point_name}")
        
        # API 요청 제한 고려
        time.sleep(0.3)
    
    # 결과를 CSV 파일로 저장
    if results:
        save_results_to_csv(results, output_filename)
    else:
        print("\n바다 근처의 낚시 포인트를 찾지 못했습니다.")

# 이 함수는 원래 코드에서 그대로 가져옴
def save_results_to_csv(results, output_filename):
    try:
        # 모든 결과의 키 수집
        all_keys = set()
        for result in results:
            all_keys.update(result.keys())
        
        fieldnames = list(all_keys)
        
        with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for result in results:
                writer.writerow(result)
        
        print(f"\n총 {len(results)}개의 바다 근처 낚시 포인트를 '{output_filename}'에 저장했습니다.")
        
    except Exception as e:
        print(f"CSV 파일 저장 중 오류 발생: {e}")

In [38]:
import requests
import csv
import time
import json
import os
from haversine import haversine
import dotenv

def load_fishing_points(filename):
    """
    CSV 파일에서 낚시 포인트 이름을 로드하는 함수
    
    Args:
        filename (str): 낚시 포인트 이름이 저장된 CSV 파일 경로
    
    Returns:
        list: 낚시 포인트 이름 리스트
    """
    fishing_points = []
    
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if 'name' in row and row['name']:
                    fishing_points.append(row['name'])
        
        print(f"{len(fishing_points)}개의 낚시 포인트 이름을 로드했습니다.")
        
    except Exception as e:
        print(f"CSV 파일 로드 중 오류 발생: {e}")
    
    return fishing_points

def get_coordinates_from_kakao(place_name, api_key):
    """
    카카오맵 API를 사용하여 장소 이름으로 좌표를 검색하는 함수
    
    Args:
        place_name (str): 검색할 장소 이름
        api_key (str): 카카오맵 API 키
    
    Returns:
        dict: 검색 결과 (이름, 위도, 경도, 주소 등)
    """
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {
        "Authorization": f"KakaoAK {api_key}"
    }
    params = {
        "query": place_name,
        "page": 1,
        "size": 15
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result.get("documents"):
                # 검색 결과 중 첫 번째 항목 사용
                first_result = result["documents"][0]
                
                return {
                    "name": place_name,
                    "latitude": float(first_result["y"]),
                    "longitude": float(first_result["x"]),
                    "address": first_result.get("address_name", ""),
                    "place_name": first_result.get("place_name", ""),
                    "category": first_result.get("category_name", ""),
                    "phone": first_result.get("phone", ""),
                    "place_url": first_result.get("place_url", "")
                }
        else:
            print(f"API 요청 실패: {response.status_code}")
    
    except Exception as e:
        print(f"{place_name} 검색 중 오류: {e}")
    
    return None

def is_near_sea(latitude, longitude, sea_coords, max_distance_km=2.0):
    """
    주어진 좌표가 바다 근처인지 확인하는 함수
    
    Args:
        latitude (float): 위도
        longitude (float): 경도
        sea_coords (list): 바다 해안선 좌표 리스트 [(lat1, lon1), (lat2, lon2), ...]
        max_distance_km (float): 바다로부터 최대 거리 (km)
    
    Returns:
        bool: 바다 근처인 경우 True, 아니면 False
    """
    point = (latitude, longitude)
    
    # 모든 해안선 좌표와 비교하여 가장 가까운 거리 계산
    min_distance = float('inf')
    
    for sea_point in sea_coords:
        distance = haversine(point, sea_point)
        min_distance = min(min_distance, distance)
        
        # 최대 거리 이내에 있으면 즉시 True 반환
        if distance <= max_distance_km:
            return True
    
    return min_distance <= max_distance_km

def load_sea_coordinates():
    """
    한국 해안선 주요 좌표를 반환하는 함수
    실제로는 더 많은 좌표가 필요하지만, 여기서는 샘플만 제공
    
    Returns:
        list: 해안선 좌표 [(위도, 경도), ...] 형태의 리스트
    """
    # 한국 주요 해안 지역 좌표 (동해, 서해, 남해)
    return [
        # 동해안
        (37.5036, 129.4327),  # 강원도 삼척
        (37.4913, 129.1813),  # 강원도 동해
        (37.7773, 128.9478),  # 강원도 강릉
        (38.2074, 128.5933),  # 강원도 속초
        (36.9932, 129.4175),  # 경북 울진
        (36.5356, 129.4454),  # 경북 영덕
        (36.0541, 129.3840),  # 경북 포항
        (35.8907, 129.5126),  # 경북 경주
        (35.1545, 129.1499),  # 부산 해운대
        
        # 남해안
        (34.8118, 127.9068),  # 경남 통영
        (34.9348, 128.0428),  # 경남 거제
        (34.7700, 127.6639),  # 전남 여수
        (34.3052, 126.5375),  # 전남 완도
        (34.6813, 125.9531),  # 전남 진도
        (34.4848, 126.2637),  # 전남 해남
        
        # 서해안
        (35.9598, 126.6014),  # 전북 군산
        (36.3285, 126.5926),  # 충남 보령
        (36.7817, 126.1494),  # 충남 태안
        (37.0255, 126.4090),  # 충남 당진
        (37.1749, 126.2357),  # 경기 화성
        (37.4462, 126.2626),  # 경기 시흥
        (37.3804, 126.5973),  # 인천 강화
        
        # 제주도
        (33.4996, 126.5312),  # 제주시
        (33.2496, 126.5628),  # 서귀포시
        (33.5219, 126.8567),  # 성산일출봉
        (33.2285, 126.2143),  # 차귀도
    ]

def process_fishing_points(input_filename, output_filename, api_key):
    """
    낚시 포인트 이름을 좌표로 변환하고 바다 근처 포인트만 필터링하는 함수
    
    Args:
        input_filename (str): 낚시 포인트 이름이 저장된 CSV 파일 경로
        output_filename (str): 결과를 저장할 CSV 파일 경로
        api_key (str): 카카오맵 API 키
    """
    # 낚시 포인트 이름 로드
    fishing_points = load_fishing_points(input_filename)
    
    if not fishing_points:
        print("처리할 낚시 포인트가 없습니다.")
        return
    
    # 해안선 좌표 로드
    sea_coords = load_sea_coordinates()
    print(f"{len(sea_coords)}개의 해안선 좌표를 로드했습니다.")
    
    # 결과 저장할 리스트
    results = []
    
    # 각 낚시 포인트에 대해 좌표 검색 및 바다 근처 확인
    for i, point_name in enumerate(fishing_points):
        print(f"[{i+1}/{len(fishing_points)}] '{point_name}' 처리 중...")
        
        # 카카오맵 API로 좌표 검색
        result = get_coordinates_from_kakao(point_name, api_key)
        
        if result:
            # 바다 근처인지 확인
            is_sea_nearby = is_near_sea(result["latitude"], result["longitude"], sea_coords)
            result["near_sea"] = is_sea_nearby
            
            if is_sea_nearby:
                print(f"  ✓ 바다 근처 포인트: {point_name} ({result['latitude']}, {result['longitude']})")
                results.append(result)
            else:
                print(f"  ✗ 바다에서 멀리 떨어짐: {point_name}")
        else:
            print(f"  ! 좌표를 찾을 수 없음: {point_name}")
        
        # API 요청 제한 고려 (초당 10회 제한)
        time.sleep(0.2)
    
    # 결과를 CSV 파일로 저장
    if results:
        try:
            # 모든 결과의 키 수집
            all_keys = set()
            for result in results:
                all_keys.update(result.keys())
            
            fieldnames = list(all_keys)
            
            with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                
                writer.writeheader()
                for result in results:
                    writer.writerow(result)
            
            print(f"\n총 {len(results)}개의 바다 근처 낚시 포인트를 '{output_filename}'에 저장했습니다.")
            
        except Exception as e:
            print(f"CSV 파일 저장 중 오류 발생: {e}")
    else:
        print("\n바다 근처의 낚시 포인트를 찾지 못했습니다.")


In [39]:
def main():
    # 입력 및 출력 파일 경로
    input_file = "fishing_points.csv"  # 낚시 포인트 이름이 저장된 CSV
    output_file = "fishing_points_coordinates.csv"  # 결과 저장할 CSV
    
    # 카카오맵 API 키 설정
    # 환경 변수에서 API 키 로드 시도
    dotenv.load_dotenv()
    kakao_api_key = os.environ.get("KAKAO_API_KEY")
    if not kakao_api_key:
        print("⚠️ 카카오맵 API 키를 설정해주세요!")
        print("1. https://developers.kakao.com 에서 앱을 등록하고 API 키를 발급받으세요.")
        print("2. 발급받은 REST API 키를 코드에 입력하거나 KAKAO_API_KEY 환경 변수로 설정하세요.")
        return
    
    # 처리 시작
    print("낚시 포인트 이름을 좌표로 변환하고 바다 근처 포인트만 필터링합니다...")
    process_fishing_points(input_file, output_file, kakao_api_key)

if __name__ == "__main__":
    main()

낚시 포인트 이름을 좌표로 변환하고 바다 근처 포인트만 필터링합니다...
1415개의 낚시 포인트 이름을 로드했습니다.
26개의 해안선 좌표를 로드했습니다.
[1/1415] '진도' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 진도
[2/1415] '어란진항' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 어란진항
[3/1415] '서망항' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 서망항
[4/1415] '갑도' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 갑도
[5/1415] '상마도' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 상마도
[6/1415] '벽파진' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 벽파진
[7/1415] '옥동리' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 옥동리
[8/1415] '진도수품' 처리 중...
  ✓ 바다 근처 포인트: 진도수품 (34.4809772653444, 126.262075945595)
[9/1415] '팽목' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 팽목
[10/1415] '하조도' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 하조도
[11/1415] '상조도' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 상조도
[12/1415] '서거차항' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 서거차항
[13/1415] '우이도항' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 우이도항
[14/1415] '흑산도' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 흑산도
[15/1415] '홍도항' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 홍도항
[16/1415] '능산도' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 능산도
[17/1415] '비금도수대' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 비금도수대
[18/1415] '원평항' 처리 중...
  ✗ 바다에서 멀리 떨어짐: 원평항
[19/1415] '북강수도' 처리 중...
  ! 좌표를 찾을 수 없음: 북강수도
[20/1415] '하태도' 처리 중...